In [ ]:
# !pip install opencv-python

In [1]:
import cv2
import torch , torchvision

In [2]:

# # define a video capture object 
# vid = cv2.VideoCapture(0) 
  
# while(True): 
      
#     # Capture the video frame 
#     # by frame 
#     ret, frame = vid.read() 
  
#     # Display the resulting frame 
#     frame = cv2.rotate(frame,cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
#     cv2.imshow('frame', frame) 
#     # the 'q' button is set as the 
#     # quitting button you may use any 
#     # desired button of your choice 
#     if cv2.waitKey(1) & 0xFF == ord('q'): 
#         break
  
# # After the loop release the cap object 
# vid.release() 
# # Destroy all the windows 
# cv2.destroyAllWindows() 

In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from matplotlib import pyplot as plt

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [5]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

In [8]:
def process_frame(frame):
    outputs = predictor(frame)
    mask = outputs['instances'].pred_masks.to('cpu').numpy()
    new_gaussian_frame = np.zeros([frame.shape[0],frame.shape[1],frame.shape[2]],dtype=np.uint8)
    
    for i in range(3):
        new_gaussian_frame[:,:,i] = frame[:,:,i]*(1 - mask[0])
    for i in range(3):
        frame[:,:,i] = frame[:,:,i]*(mask[0])
        
    frame = frame + cv2.GaussianBlur(new_gaussian_frame,(51,51),0)
    return frame

In [9]:
# define a video capture object 
vid = cv2.VideoCapture(0) 
  
while(True): 
      
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
  
    # Display the resulting frame 
    frame = cv2.rotate(frame,cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
    # process the frame
    frame = process_frame(frame)
    cv2.imshow('frame', frame) 
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

In [ ]:
# outputs = predictor(frame)
# mask = outputs['instances'].pred_masks.to('cpu').numpy()
# # for i in range(3):
# #     frame[:,:,i] = frame[:,:,i]*(mask[0])

In [ ]:
# new_gaussian_frame = np.zeros([frame.shape[0],frame.shape[1],frame.shape[2]],dtype=np.uint8)
# # blur = cv2.GaussianBlur(resized_frame,(41,41),0)

# for i in range(3):
#     new_gaussian_frame[:,:,i] = frame[:,:,i]*(1 - mask[0])
# for i in range(3):
#     frame[:,:,i] = frame[:,:,i]*(mask[0])
# frame = frame + new_gaussian_frame
# plt.imshow(cv2.GaussianBlur(new_gaussian_frame,(7,7),0))